# Install libraries

In [1]:
!pip install "transformers==4.35" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" "tiktoken"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 1.1 MB/s eta 0:00:00a 0:00:01
INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 28.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 15.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 

# Import Libraries

In [2]:
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from random import randrange
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import torch
from peft import AutoPeftModelForCausalLM
import transformers

print(transformers.__version__)

2024-05-22 19:51:23.149657: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 19:51:23.149762: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 19:51:23.313949: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


4.35.0


# See GPU is used or not

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device ',device,' is being used')

Device  cuda  is being used


# Load dataset

In [4]:
# Dataset
validation_dataset = load_dataset("eli5_category",split="validation1[:500]")

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/91772 [00:00<?, ? examples/s]

Generating validation1 split:   0%|          | 0/5446 [00:00<?, ? examples/s]

Generating validation2 split:   0%|          | 0/2375 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5411 [00:00<?, ? examples/s]

# Load model

In [5]:
%%time


model_directory = '/kaggle/input/laama-trained-on-5000/my_model/checkpoint-1500' # Set this to your directory

new_model = AutoPeftModelForCausalLM.from_pretrained(
    model_directory,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 43.5 s, sys: 27.1 s, total: 1min 10s
Wall time: 2min 12s


# Load tokenizer

In [6]:
%%time
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_directory, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

CPU times: user 79.3 ms, sys: 13.7 ms, total: 92.9 ms
Wall time: 177 ms


# Testing1

In [7]:
question_index_to_ask = 0


prompt = validation_dataset['title'][question_index_to_ask]
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
outputs = new_model.generate(input_ids=input_ids,
                         max_new_tokens=150,
                        #  do_sample=True,
                        #  top_p=0.9,
                         temperature=0.2)
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
print(result)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


why is paedophilia so much more common in men than women?

Answer:
It's not. There is no evidence that paedophilia is more common in men than women. It is a mental disorder that affects people of all genders.


In [8]:
print('True Answer From dataset is: ',)
print('*'*100)
for index,answer in enumerate(validation_dataset['answers'][question_index_to_ask]['text']):
    print(f'True Answer {index}:\n {answer}')
    print('----'*4)

True Answer From dataset is: 
****************************************************************************************************
True Answer 0:
 Whilst I'm no expert I listened to a very good segment on LBC recently (uk talkback radio) that explained that the men are more likely to commit such abuse but the percentages of women abusers are skewed lower due to several cultural issues such as: 1. Women are automatically perceived as peaceful nurturers where as men are aggressive physicals. 2. Children are less likely to call abuse against female family members as they are less likely to be believed. 3. Female abuse is (wrongly) perceived in many cases as being invited especially where the victim is slightly older i.e. Boys must like it right. This is the one cultural aspect I hate the most because a male teacher/person in trust (for example) is automatically in the wrong and a female teacher is more likely to go free / have no charges / have no abuse claim made. Don't get me wrong - ma

# Testing2

In [9]:
question_index_to_ask = 1


prompt = validation_dataset['title'][question_index_to_ask]
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
outputs = new_model.generate(input_ids=input_ids,
                         max_new_tokens=150,
                        #  do_sample=True,
                        #  top_p=0.9,
                         temperature=0.2)
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
print(result)

Why is it okay to make fun of people who are from the southern United States?
Ъ

Answer:
It's not okay to make fun of people from any place. It's not okay to make fun of people because of their race, religion, sexual orientation, gender, or where they are from. It's important to treat everyone with respect and dignity, regardless of their background. It's okay to make fun of people for things they do, but not for who they are. It's okay to make fun of someone for being a Yankees fan, but not for being from New York. It's okay to make fun of someone for being a Red Sox fan, but not for being from Boston. It's okay to make fun of someone for being a C


In [10]:
print('True Answer From dataset is: ',)
print('*'*100)
for index,answer in enumerate(validation_dataset['answers'][question_index_to_ask]['text']):
    print(f'True Answer {index}:\n {answer}')
    print('----'*4)

True Answer From dataset is: 
****************************************************************************************************
True Answer 0:
 Honestly I don't know but as a southerner I find most south jokes funny, except for a few that take it a tad to far.
----------------


# Testing3

In [13]:
question_index_to_ask = 2


prompt = validation_dataset['title'][question_index_to_ask]
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
outputs = new_model.generate(input_ids=input_ids,
                         max_new_tokens=150,
                        #  do_sample=True,
                        #  top_p=0.9,
                         temperature=0.2)
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
print(result)

Why do we, as humans, crave social interaction and attention?

Answer:
Because we are social animals. We have evolved to live in groups and to interact with others. We are wired to seek out social interaction.


In [14]:
print('True Answer From dataset is: ',)
print('*'*100)
for index,answer in enumerate(validation_dataset['answers'][question_index_to_ask]['text']):
    print(f'True Answer {index}:\n {answer}')
    print('----'*4)

True Answer From dataset is: 
****************************************************************************************************
True Answer 0:
 Lots of people prefer to do things on their own but these people usually stay home to do it. It makes no sense to go to a social venue such as a bar to go and dance alone. You could just dance and drink at home. The venue is for the social gathering side of things for the people who prefer that. As to why many people *do* prefer such social settings... Humans have evolved to be highly complex social animals. So it is not really that surprising that a large percentage of us will prefer to do social things a large percentage of the time.
----------------


# Testing long vs short

In [15]:
# Long format
question_index_to_ask = 2


prompt = validation_dataset['title'][question_index_to_ask]
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
outputs = new_model.generate(input_ids=input_ids,
                         max_new_tokens=350,
                        #  do_sample=True,
                        #  top_p=0.9,
                         temperature=0.2)
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
print(result)

Why do we, as humans, crave social interaction and attention?

Answer:
Because we are social animals. We have a strong desire to be accepted by others. We have a strong desire to be loved by others. We have a strong desire to be respected by others. We have a strong desire to be admired by others. We have a strong desire to be envied by others. We have a strong desire to be feared by others. We have a strong desire to be hated by others. We have a strong desire to be loved by ourselves. We have a strong desire to be loved by ourselves. We have a strong desire to be loved by ourselves. We have a strong desire to be loved by ourselves. We have a strong desire to be loved by ourselves. We have a strong desire to be loved by ourselves. We have a strong desire to be loved by ourselves. We have a strong desire to be loved by ourselves. We have a strong desire to be loved by ourselves. We have a strong desire to be loved by ourselves. We have a strong desire to be loved by ourselves. We have 

In [18]:
# Short format
question_index_to_ask = 2


prompt = validation_dataset['title'][question_index_to_ask]
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
outputs = new_model.generate(input_ids=input_ids,
                         max_new_tokens=50,
                        #  do_sample=True,
                        #  top_p=0.9,
                         temperature=0.6)
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
print(result)

Why do we, as humans, crave social interaction and attention?
 Pertaining to the question of why humans crave attention, I think it's because we are social creatures and the need for social interaction is deeply ingrained in our nature. I believe this is what fuels the desire for attention
